# LLM-as-Judge: Classify if user post is prediction or not

In [ ]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [ ]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

- Each data to load is saved in individual file. 
- Before idea of saving all data to one file. 
- Save to same file for many reasons.
    - [Reason 1](https://github.com/ufdatastudio/predictions/issues/71)
    - Reason 2: I think in previous step, I save by appending, then save as new file. Then below, I load all csvs when if previous sentence is correct, I should load only last csv.

In [ ]:
def load_open_measures_data(notebook_dir: str, site: str) -> pd.DataFrame:
    """Load individual csv files of data
    
    Parameters
    ----------
    notebook_dir : str
        The location of this notebook, so we can save files using relative paths
    site : str
        Source of the data ('tiktok', 'bluesky', 'truth social', 'llm generated', etc).
    """

    base_path = os.path.join(notebook_dir, '../', 'data/open_measures/', f'{site}/')
    files = os.listdir(base_path)

    dfs = []

    for file in tqdm(files):
        print(file)
        full_path = os.path.join(base_path, file)
        df = DataProcessing.load_from_file(full_path, 'csv')
        dfs.append(df)

    return dfs

In [ ]:
bluesky_dfs = load_open_measures_data(notebook_dir, 'bluesky_raw_data')
bluesky_dfs

In [ ]:
truth_social_dfs = load_open_measures_data(notebook_dir, 'truth_social_raw_data')
truth_social_dfs

## Data Processing

In [ ]:
bluesky_df = DataProcessing.concat_dfs(bluesky_dfs)
print(f"Bluesky before duplicates: {bluesky_df.duplicated().any()}")
bluesky_df = DataProcessing.remove_duplicates(bluesky_df)
print(f"Bluesky after duplicates: {bluesky_df.duplicated().any()}\n")
bluesky_df.rename(columns={"Unnamed: 0": "data_range_per_file"}, inplace=True)

truth_social_df = DataProcessing.concat_dfs(truth_social_dfs)
print(f"Truth Social before duplicates: {truth_social_df.duplicated().any()}")
truth_social_df = DataProcessing.remove_duplicates(truth_social_df)
print(f"Truth Social after duplicates: {truth_social_df.duplicated().any()}\n")
truth_social_df.rename(columns={"Unnamed: 0": "data_range_per_file"}, inplace=True)
truth_social_df['text'] = truth_social_df['content_cleaned']

In [ ]:
len(bluesky_df), len(truth_social_df)

## Detect Prediction Label with LLM + Majority Vote

In [ ]:
tgmf = TextGenerationModelFactory()

# Groq Cloud (https://console.groq.com/docs/overview)
gemma_29b_generation_model = tgmf.create_instance('gemma2-9b-it') 
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
# llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

# models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model, llama_guard_4_12b_generation_model]

# models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model]
models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_guard_4_12b_generation_model]

In [ ]:
def count_label(current_label, label_to_counts):
    """
    Given a sentence, use any LLM from text_generation_models to detect if it's a prediction or not.

    Parameters
    ----------
    current_label : pd.DataFrame
        Label from specific LLM on if it's a prediction or not
    label_to_counts : dict
        Data with the model_label : count

    Returns
    -------
    None
        Only update dictionary: label_to_counts
    
    
    """
    current_label = current_label.strip()
    # print(f"\tBase label_to_counts: {label_to_counts}")
    if current_label in label_to_counts.keys():
        old_count = label_to_counts[current_label]
        
        update_count = old_count + 1
        label_to_counts[current_label] = update_count
        # print(f"\tUpdated label_to_counts: {label_to_counts}")
    else:
        label_to_counts[current_label] = 1
        # print(f"\tUpdated label_to_counts: {label_to_counts}")

In [ ]:
def get_majority_vote(data):
    """
    Given a dictionary, compute majority vote

    Parameters
    ----------
    data : dict
        Data with the model_label : count

    Returns
    -------
    str
        The label of the mode

    """
    mode = max(data.values()) 
    for key, val in data.items():
        if val == mode:
            if mode == 1: 
                majority_vote_label = 'None as all are different labels'
            else:
                majority_vote_label = key

    return majority_vote_label

Update prompt with prediction defintion?

- Prediction properties {}  --- Copy from another notebook
- Prediction is in future tense
- Prediction examples: 
    1. "2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports" as p_d = 2016, p_o = Super Bowl (odds) and co-favorites, p_t = Patriots
    2. "Is there a point to the rest of the #NFL season...or should we just get ready to cheer against the #Patriots in the Super Bowl?" because p_t = #Patriots p_o = in the Super Bowl
    3. "Utilitarianism's current choice for 2018 FIFA #WorldCup championship: Mexico.\n\nWith many caveats.\n\nblog.practicalethics.ox.ac.uk/2018/06/the-ut…" as p_s = Utilitarianism, p_t = FIFA #WorldCup championship, p_o = Mexico in 2018 FIFA #WorldCup championship p_d = 2018s
- Non-prediction examples:
    1. "Forgot to post this today. From 1982, Brimstone prepares for the Super Bowl. Disappointment follows" as not future tense.
    2. "Where do Reid and Mahomes sit among QB/coaching duos for playoffs wins?" as not future tense, no p_o


In [ ]:
def detect_predictions_with_llms(df: pd.DataFrame, notebook_dir: str, site: str):
    """
    Given a sentence, use any LLM from text_generation_models to detect if it's a prediction or not.

    Parameters
    ----------
    df : pd.DataFrame
        Data with the sentences we want to label
    notebook_dir : str
        The location of this notebook, so we can save files using relative paths
    site : str
        Source of the data ('tiktok', 'bluesky', 'truth social', 'llm generated', etc).

    Returns
    -------
    list[dict]
        sentence, model, label, meta_data, and majority vote label
    
    
    """
    labels = []
    batch_size = 50
    show_data = 1

    sentences = DataProcessing.df_to_list(df, col='text')
    # print(sentences)
    meta_data_df = df.drop(columns=['text'])

    
    for batch_idx in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[batch_idx:batch_idx+batch_size]
        for sentences_idx, sentence in enumerate(batch):
            label_to_count = {}
            # sentence = batch[sentences_idx]
            sent_meta_data_series = meta_data_df.iloc[batch_idx + sentences_idx]
            sent_meta_data = sent_meta_data_series.to_dict()
            if show_data <=3: 
                # print(f"\>>>Sentence: {sentence} --- {site}\nMetadata: {sent_meta_data}")
                # print(f"\n>>>Sentence: {sentence} --- {site}")
                print(f">>>Sentence: {type(sentence)}, {sentence}")
                show_data += 1

            prompt = f"""
                My work deals with identifying predictions from humans (namely Human Predictions or HP). As of now, I'm gathering textual data only, so HP comes in the form of text-based predictions. HPs can be found in the domains of finance, health, policy, weather, sports, and miscellaneous. Some examples in sports are which teams will make the playoffs, which two teams will be in the championship game, coupled with which of these two teams will actually win the championship. Given this sentence ({sentence}), classify if the sentence is a prediction, not a prediction, or not enough information/neutral. Do not explain or provide any other details. 
            
                Examples:
                (1) Sentence: The touchdown rate at the Miami Dolphins will fall in 2020 of October. Label: prediction
                (2) Sentence: Manchester City vs. Watford: Predicted lineups, team news and score prediction ⚽️. Label: not a prediction
                (3) Sentence: The Patriots are looking strong this season. Label: not enough information/neutral
                
                Remember, your responses are discrete corresponding to the states in the list:[prediction, not a prediction, not enough information/neutral].
            """
            
            sentence_labels = []
            # print(f"Prompt: {prompt}")
            for model in models:  
                input_prompt = model.user(prompt)
                # print(input_prompt)  
                
                raw_text_llm_generation = model.chat_completion([input_prompt])
                # print(raw_text_llm_generation)
                # print("====================================")
                for line in raw_text_llm_generation.split("\n"):
                    # print(line)
                    if line.strip():
                        label = line.strip()
                        # print(f"\tModel -> Label: {model.__name__()} -> {label}")
                        count_label(label, label_to_count)

                        sentence_labels.append({
                            "sentence": sentence,
                            "model": model.__name__(),
                            "label": label,
                            "meta_data": sent_meta_data
                        })

            majority_vote_label = get_majority_vote(label_to_count)
            # print(f"\t\tMajority vote: {majority_vote_label}")

            # Attach majority vote to each model's label
            for entry in sentence_labels:
                entry["majority_vote"] = majority_vote_label
                labels.append(entry)

        print()
        # print(f"Labels: {labels}")
        save_dir = os.path.dirname(notebook_dir)
        # print(f"Site: {site}")
        save_dir = os.path.join(save_dir, 'data', 'open_measures', f"{site}_all_llm_labels")
        # print(f"\n\tLabels: {labels}")
        DataProcessing.save_to_file(labels, save_dir, site)
    return labels

In [ ]:
updated_column_names={'sentence': 'Text', 'model': 'Model', 'label': 'Label', 'meta_data': 'Meta Data', 'majority_vote': 'Majority Vote'}

In [ ]:
len(bluesky_df), len(truth_social_df)

- Need to fix save path. It works, but just need to recogfigure it.

In [24]:
bluesky_site = 'bluesky'
llms_generated_for_bluesocial_comments = detect_predictions_with_llms(bluesky_df[:33], notebook_dir, bluesky_site)
llms_generated_for_bluesocial_comments_df = pd.DataFrame(llms_generated_for_bluesocial_comments).rename(columns=updated_column_names)
llms_generated_for_bluesocial_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

>>>Sentence: <class 'str'>, Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.

the-afc.com/u-16-champions…
>>>Sentence: <class 'str'>, Sad! AFC bans Nepal from U16 Asian Championship 2016 & fines $14.5k for playing overage players in qualifiers. #football
>>>Sentence: <class 'str'>, 2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports


100%|██████████| 1/1 [01:16<00:00, 76.44s/it]


The json file is saving at: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/open_measures/bluesky_all_llm_labels/bluesky-v10.json
Saved to: 
	/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/open_measures/bluesky_all_llm_labels/bluesky-v10.json


,Text,Model,Label,Meta Data,Majority Vote
0,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,gemma2-9b-it,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",None as all are different labels
1,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,llama-3.1-8b-instant,prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",None as all are different labels
2,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",None as all are different labels
3,Sad! AFC bans Nepal from U16 Asian Championship 2016 & fines $14.5k for playing overage players in qualifiers. #football,gemma2-9b-it,not a prediction,"{'data_range_per_file': 1, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjko

In [ ]:
len(truth_social_df)

In [ ]:
truth_social_df.dropna(subset=['text'], inplace=True)

In [ ]:
len(truth_social_df)

- Need to fix save path. It works, but just need to recogfigure it.
- Index is off for truth_social.

In [23]:
truth_social_site = 'truth_social'
llms_generated_for_true_social_comments = detect_predictions_with_llms(truth_social_df[:33], notebook_dir, truth_social_site)
llms_generated_for_true_social_comments_df = pd.DataFrame(llms_generated_for_true_social_comments).rename(columns=updated_column_names)
llms_generated_for_true_social_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

>>>Sentence: <class 'str'>, I’m so tired of  hearing about Colin Kaepernick he’s not good enough to win a Super Bowl he’s not good enough to make the playoffs has nothing to do with his skin color he sucks also why would he want to be a slave didn’t he say NFL was a slave
>>>Sentence: <class 'str'>, Burrow: Bengals 'know what it takes' now to win https://www.espn.com/nfl/story/_/id/33934040/joe-burrow-says-cincinnati-bengals-know-takes-now-win-run-super-bowl
>>>Sentence: <class 'str'>, https://www.instagram.com/reel/Cha-AWmst5v/?igshid=YmMyMTA2M2Y=


100%|██████████| 1/1 [00:53<00:00, 53.62s/it]


The json file is saving at: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/open_measures/truth_social_all_llm_labels/truth_social-v1.json
Saved to: 
	/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/open_measures/truth_social_all_llm_labels/truth_social-v1.json


,Text,Model,Label,Meta Data,Majority Vote
0,I’m so tired of hearing about Colin Kaepernick he’s not good enough to win a Super Bowl he’s not good enough to make the playoffs has nothing to do with his skin color he sucks also why would he want to be a slave didn’t he say NFL was a slave,gemma2-9b-it,not a prediction,"{'data_range_per_file': 0, '$type': nan, 'author': nan, 'authorProfile': nan, 'cid': nan, 'createdAt': nan, 'embed': nan, 'entities': nan, 'facets': nan, 'labels': nan, 'langs': nan, 'reply': nan, 'tags': '[]', 'uri': 'https://truthsocial.com/@ImBackBitchs/108189027673686555', 'lastseents': '2025-07-25T20:05:45.799767', 'embeds': nan, 'Query Params': '{'term': '(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)', 'limit': 2000, 'site': 'truth_social', 'since': '2015-09-05', 'until': '2025-02-09', 'querytype': 'query_string', 'model': 'user'}', 'Site': 'truth_social', 'account': '{'acct': 'ImBackBitchs', 'display_name': 'Just Another White Guy', 'id': '107911908216773717', 'username': 'ImBackBitchs'}', 'bookmarked': False, 'card': nan, 'collected_by': 'smat-scrapy-crawlers', 'content': '<...",None as all are different labels
1,I’m so tired of hearing about Colin Kaepernick he’s not good enough to win a Super Bowl he’s not good enough to make the playoffs has nothing to do with his skin color he sucks also why would he want to be a slave didn’t he say NFL was a slave,llama-3.1-8b-instant,prediction,"{'data_range_per_file': 0, '$type': nan, 'author': nan, 'authorProfile': nan, 'cid': nan, 'createdAt': nan, 'embed': nan, 'entities': nan, 'facets': nan, 'labels': nan, 'langs': nan, 'reply': nan, 'tags': '[]', 'uri': 'https://truthsocial.com/@ImBackBitchs/108189027673686555', 'lastseents': '2025-07-25T20:05:45.799767', 'embeds': nan, 'Query Params': '{'term': '(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)', 'limit': 2000, 'site': 'truth_social', 'since': '2015-09-05', 'until': '2025-02-09', 'querytype': 'query_string', 'model': 'user'}', 'Site': 'truth_social', 'account': '{'acct': 'ImBackBitchs', 'display_name': 'Just Another White Guy', 'id': '107911908216773717', 'username': 'ImBackBitchs'}', 'bookmarked': False, 'card': nan, 'collected_by': 'smat-scrapy-crawlers', 'content': '<...",None as all are different labels
2,I’m so tired of hearing about Colin Kaepernick he’s not good enough to win a Super Bowl he’s not good enough to make the playoffs has nothing to do with his skin color he sucks also why would he want to be a slave didn’t he say NFL was a slave,meta-llama/llama-guard-4-12b,unsafe,"{'data_range_per_file': 0, '$type': nan, 'author': nan, 'authorProfile': nan, 'cid': nan, 'createdAt': nan, 'embed': nan, 'entities': nan, 'facets': nan, 'labels': nan, 'langs': nan, 'reply': nan, 'tags': '[]', 'uri': 'https://truthsocial.com/@ImBackBitchs/108189027673686555', 'lastseents': '2025-07-25T20:05:45.799767', 'embeds': nan, 'Query Params': '{'term': '(NFL OR nfl) AND (playoffs) AND (Super Bowl LIX)', 'limit': 2000, 'site': 'truth_social', 'since': '2015-09-05', 'until': '2025-02-09', 'querytype': 'query_string', 'model': 'user'}', 'Site': 'truth_social', 'account': '{'acct': 'ImBackBitchs', 'display_name': 'Just Another White Guy', 'id': '107911908216773717', 'username': 'ImBackBitchs'}', 'bookmarked': False, 'card': nan, 'collected_by': 'smat-scrapy-crawlers', 'content': '<...",None as all are different labels
3,I’m so tired of hearing about Colin Kaepernick he’s not good enough to win a Super Bowl he’s not good enough to make the playoffs has nothing to do with his skin color he sucks also why would he want to be a slave didn’t he say NFL was a slave,meta-llama/llama-guard-4-12b,S10,"{'data_range_per_file': 0, '$type': nan, 'author': nan, 'authorProfile': nan, 'cid': nan, 'createdAt': nan, 'embed': nan, 'entities': nan, 'facets': nan, 'labels': nan, 'langs': nan, 'reply': nan, 'tags': '[]', 'uri': 'https://truthsocial.com/@ImBackBitchs/108189027673686555', 'lastseents': '2025-07-25T20:05:45.799767', 'embeds': nan, 